<a href="https://colab.research.google.com/github/shirleyrutgers/DataVis/blob/main/database.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Database
Business Application and LQ Private Industry

Step 1. Download file from online and save in colab

Data

Business applications
https://docs.google.com/spreadsheets/d/e/2PACX-1vRl9ztuLbIaUzAFe1YgGB7IRhZL4P5W3dL3XeSCna8RXFdJAfIya78QR2otbTWE4w/pub?gid=13513596&single=true&output=csv

Industry Location Quotient
https://docs.google.com/spreadsheets/d/e/2PACX-1vRxQsQ5FEDOhsbuD4U76nKN8Ky_dZX6AfLQMqtqdJHHlz3guHAKUBzYb_cBfzEkaw/pub?gid=1931554944&single=true&output=csv


In [1]:
#---------------------------SETUP----------------------------------
#get useful libraries
import time, os, sys, re #basics
import zipfile, json, datetime, string   #string for annotating points in scatter
import numpy as np #basic math
from statistics import * #stats

import matplotlib.pyplot as plt #import pylab as plt #apparently discouraged now:
 #https://stackoverflow.com/questions/11469336/what-is-the-difference-between-pylab-and-pyplot
 #https://www.tutorialspoint.com/matplotlib/matplotlib_pylab_module.htm

import pandas as pd
import pandas_datareader as pdr
from pandas_datareader import wb
from pandas.io.formats.style import Styler
#s4 = Styler(df4, uuid_len=0, cell_ids=False)

import urllib  #weird, guess need to have os and pandas imported for this to work  %TODO/LATER ditch it, its weird anyway, just use wget/curl

from google.colab import files

#import webbrowser

import seaborn as sns

from google.colab import data_table
data_table.enable_dataframe_formatter() #this enables spreadsheet view upon calling dataframe (without() )

#tricks how to extend notebook functionality
#https://coderzcolumn.com/tutorials/python/list-of-useful-magic-commands-in-jupyter-notebook-lab
#will display all output not just last command
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

###magics: https://ipython.readthedocs.io/en/stable/interactive/magics.html
#most essential setup for vis: it does affect vis! careful!! stick with inline, maybe notebook; others mostly for non-notebook, eg spyder environ
#https://jakevdp.github.io/PythonDataScienceHandbook/04.00-introduction-to-matplotlib.html recomends *inline*!
#show current one:
#%matplotlib
#%matplotlib --list
#interactive plots:
#%matplotlib notebook
#static images of your plot:
%matplotlib inline
#this one and other magics (btw default is probably agg)
#%matplotlib nbagg
##https://www.marktechpost.com/2023/10/20/6-magic-commands-for-jupyter-notebooks-in-python-data-science/
#%%latex
#%ai
#%run
#%writefile
#%history -n

###themes/styles: https://matplotlib.org/stable/gallery/style_sheets/style_sheets_reference.html
#https://jakevdp.github.io/PythonDataScienceHandbook/04.11-settings-and-stylesheets.html
#https://matplotlib.org/stable/tutorials/introductory/customizing.html
#here more about art and style than under the hood functionality as with magics, explore and experiment
#many may find 'default' or seaborn ones more pleasing; my fav 'classic' is back from 90s ;)
#plt.style.available #list available styles :) may install more
plt.style.use('default') # more delicate subtle than classic
#plt.style.use('classic')  #  'seaborn-whitegrid' 'seaborn-white' 'seaborn-poster'
# btw: magics v theme/style sequence matters, eg if i specify classic style before inline magic, i wouldnt get grey bounding box im getting

#sometimes have to install library which you get from https://pypi.org/
#!pip install geopandas

In [19]:
# Read Database 1, Business Application, from URL CSV, from row 3
business = pd.read_csv(
    'https://docs.google.com/spreadsheets/d/e/2PACX-1vRl9ztuLbIaUzAFe1YgGB7IRhZL4P5W3dL3XeSCna8RXFdJAfIya78QR2otbTWE4w/pub?gid=13513596&single=true&output=csv',
    skiprows=2
)

# Review
business.head()

,State,County,County Code,fipstate,fipscty,BA2005,BA2006,BA2007,BA2008,BA2009,...,BA2014,BA2015,BA2016,BA2017,BA2018,BA2019,BA2020,BA2021,BA2022,BA2023
0,AL,Autauga County,1001,1,1,335,348,348,367,288,...,290,291,294,313,384,394,559,776,707,703
1,AL,Baldwin County,1003,1,3,1986,2101,1975,1721,1505,...,1575,1735,1858,2114,2104,2317,2697,3407,3694,3855
2,AL,Barbour County,1005,1,5,138,179,156,139,133,...,120,129,122,114,113,135,230,334,266,421
3,AL,Bibb County,1007,1,7,118,145,117,98,98,...,101,78,86,79,105,86,165,199,158,173
4,AL,Blount County,1009,1,9,315,323,297,306,247,...,228,218,254,281,299,312,344,473,428,513


In [20]:
#Delete columns from 2005 to 2022
cols_to_drop = [
    'BA2005', 'BA2006', 'BA2007', 'BA2008', 'BA2009', 'BA2010',
    'BA2011', 'BA2012', 'BA2013', 'BA2014', 'BA2015', 'BA2016',
    'BA2017', 'BA2018', 'BA2019', 'BA2020', 'BA2021', 'BA2022'
]

# Delete
business = business.drop(columns=cols_to_drop)

# Review
business.head()

,State,County,County Code,fipstate,fipscty,BA2023
0,AL,Autauga County,1001,1,1,703
1,AL,Baldwin County,1003,1,3,3855
2,AL,Barbour County,1005,1,5,421
3,AL,Bibb County,1007,1,7,173
4,AL,Blount County,1009,1,9,513


In [18]:
#Read Industry Data from URL CSV
industry=pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vRxQsQ5FEDOhsbuD4U76nKN8Ky_dZX6AfLQMqtqdJHHlz3guHAKUBzYb_cBfzEkaw/pub?gid=1931554944&single=true&output=csv')
# Review
industry.head()

<ipython-input-18-e9085c55af1a>:2: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  industry=pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vRxQsQ5FEDOhsbuD4U76nKN8Ky_dZX6AfLQMqtqdJHHlz3guHAKUBzYb_cBfzEkaw/pub?gid=1931554944&single=true&output=csv')


,Area\nCode,St,Cnty,Own,NAICS,Year,Qtr,Area Type,St Name,Area,Ownership,Industry,Annual Average Status Code,Annual Average Establishment Count,Annual Average Employment,Annual Total Wages,Annual Average Weekly Wage,Annual Average Pay,Employment Location Quotient Relative to U.S.,Total Wage Location Quotient Relative to U.S.
0,US000,US,0.0,0,10,2023,A,Nation,NaN,U.S. TOTAL,Total Covered,"10 Total, all industries",NaN,"11,866,306","153,140,899","11,081,267,615,470","1,392","72,360",1.0,1.0
1,US000,US,0.0,1,10,2023,A,Nation,NaN,U.S. TOTAL,Federal Government,"10 Total, all industries",NaN,"61,310","2,929,938","286,764,744,238","1,882","97,874",1.0,1.0
2,US000,US,0.0,2,10,2023,A,Nation,NaN,U.S. TOTAL,State Government,"10 Total, all industries",NaN,"72,215","4,658,153","356,315,881,354","1,471","76,493",1.0,1.0
3,US000,US,0.0,3,10,2023,A,Nation,NaN,U.S. TOTAL,Local Government,"10 Total, all industries",NaN,"171,844","14,263,127","905,493,894,970","1,221","63,485",1.0,1.0
4,US000,US,0.0,5,10,2023,A,Nation,NaN,U.S. TOTAL,Private,"10 Total, all industries",NaN,"11,560,938","131,289,681","9,532,693,094,908","1,396","72,608",1.0,1.0


In [16]:
#Cleanning the database> industry
#Rename the column Area
industry = industry.rename(columns={"Area": "County"})
#Rename the column Area Code
industry = industry.rename(columns={"Area Code": "County Code"})

In [17]:
# Filter> County, industry Private Ownership, and Total Industry, delete "Unknow Or Undefined"
priv= industry[
    (industry["Area Type"] == "County") &
    (industry["Ownership"] == "Private") &
    (industry["Industry"] == "10 Total, all industries") &
    (~industry["County"].str.contains("Unknown Or Undefined", na=False))
]

# Display the filtered results
priv.head()

,Area\nCode,St,Cnty,Own,NAICS,Year,Qtr,Area Type,St Name,County,Ownership,Industry,Annual Average Status Code,Annual Average Establishment Count,Annual Average Employment,Annual Total Wages,Annual Average Weekly Wage,Annual Average Pay,Employment Location Quotient Relative to U.S.,Total Wage Location Quotient Relative to U.S.
40,01001,01,1.0,5,10,2023,A,County,Alabama,"Autauga County, Alabama",Private,"10 Total, all industries",NaN,"1,011","9,330","454,329,752",936,"48,696",0.92,0.89
57,01003,01,3.0,5,10,2023,A,County,Alabama,"Baldwin County, Alabama",Private,"10 Total, all industries",NaN,"8,137","71,867","3,473,350,164",929,"48,330",1.01,1.00
75,01005,01,5.0,5,10,2023,A,County,Alabama,"Barbour County, Alabama",Private,"10 Total, all industries",NaN,533,"6,227","264,209,398",816,"42,433",0.94,0.91
92,01007,01,7.0,5,10,2023,A,County,Alabama,"Bibb County, Alabama",Private,"10 Total, all industries",NaN,380,"3,529","187,928,363","1,024","53,248",0.85,0.88
109,01009,01,9.0,5,10,2023,A,County,Alabama,"Blount County, Alabama",Private,"10 Total, all industries",NaN,867,"6,734","293,597,353",839,"43,602",0.89,0.88
